In [1]:
# !pip install -r requirements.txt
## Base of notebooks
%load_ext autoreload
%autoreload 2
import tensorflow as tf

In [2]:
from datetime import datetime

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
img_size = 128
task  = 'multiclass-b20mg-inc2'

In [3]:
import sys 
sys.path.append('..')

from utils import *
import models

In [4]:
base_data = stack_images(im_size=img_size, directory= "C:\\Users\\baran.tanriverdi\\Final Data")

100%|██████████| 43/43 [00:06<00:00,  6.50it/s]


Al for 5 Cannot found!


100%|██████████| 150/150 [00:23<00:00,  6.44it/s]

Fe2 : 51
Fe5 : 49
Fe10 : 100
Fe20 : 161
Fe200 : 153
Al2 : 43
Al10 : 16
Al20 : 150
Al200 : 136
Pb2 : 53
Pb5 : 50
Pb10 : 97
Pb20 : 150
Pb200 : 150
dict_keys(['Fe2', 'Fe5', 'Fe10', 'Fe20', 'Fe200', 'Al2', 'Al10', 'Al20', 'Al200', 'Pb2', 'Pb5', 'Pb10', 'Pb20', 'Pb200'])


In [5]:
np.asarray(base_data['Fe5']).shape

(49, 128, 128, 3)

In [6]:
# Configyure for current experiment

x1 = []
for imset in [
    base_data['Fe2'],
    base_data['Fe5'], 
    base_data['Fe10'], 
    base_data['Fe20']]:
    for im in imset:
        x1.append(im)
print(np.asarray(x1).shape)

x2 = []
for imset in [
    base_data['Al2'],
    base_data['Al10'], 
    base_data['Al20'], 
]:
    for im in imset:
        x2.append(im)
print(np.asarray(x2).shape)
        
x3 = []
for imset in [
    base_data['Pb2'],
    base_data['Pb10'], 
    base_data['Pb20']]:
    for im in imset:
        x3.append(im)
print(np.asarray(x3).shape)

imlist = []
one_hot_vector = np.eye(3)
for x in x1:
    imlist.append([x, one_hot_vector[0]])
for x in x2:
    imlist.append([x, one_hot_vector[1]])  
for x in x3:
    imlist.append([x, one_hot_vector[2]]) 
    
print(len(imlist), len(x1), len(x2), len(x3))

data = np.asarray(imlist)

np.random.shuffle(data)

x = np.asarray([i[0] / 255.0 for i in data])
y = np.asarray([i[1] for i in data])

print(len(x), len(y))
boundary = int(len(x) * 0.75)

train_x = x[:boundary]
train_y = y[:boundary]
test_x = x[boundary:]
test_y = y[boundary:]

print(len(train_x), len(test_x))

(361, 128, 128, 3)
(209, 128, 128, 3)
(300, 128, 128, 3)
870 361 209 300


C:\Users\baran.tanriverdi\AppData\Local\Temp\ipykernel_7232\1896097817.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(imlist)


870 870
652 218


In [8]:
histories = []
models_ = [
    models.unet(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    models.vgg5_block(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.vgg16(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.vgg19(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.resnet50(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.deeplabv3plus(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.nasnet_mobile(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.inception_resnetv2(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.double_unet(input_shape=(img_size, img_size, 3), number_of_classes=3)
]

for model in tqdm(models_):
    MODEL_NAME = f"{date}_{model.name}_{img_size}_{task}"
    print(MODEL_NAME)
    history =model.fit(train_x, train_y, batch_size=10,
                    validation_split=0.2,
                    epochs=25, 
                    #callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,),
                    verbose = 1)
    histories.append(history)

    save_logs(history, MODEL_NAME)
        #save m-odel
    tf.keras.models.save_model(
        model,
        f'./models/{task}/{MODEL_NAME}.h5')
           

Model: "vgg5_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_35 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_36 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_37 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                    

  0%|          | 0/2 [00:00<?, ?it/s]

2022-04-07_unet_128_multiclass-b20mg-inc2
Epoch 1/25
53/53 [==============================] - 2695s 50s/step - loss: 324.2210 - accuracy: 0.4664 - f1_score: 0.2584 - cohen_kappa: 0.1767 - MatthewsCorrelationCoefficient: 0.1782 - auc: 0.6146 - val_loss: 0.9227 - val_accuracy: 0.5496 - val_f1_score: 0.2554 - val_cohen_kappa: 0.3095 - val_MatthewsCorrelationCoefficient: 0.3145 - val_auc: 0.7728
Epoch 2/25
53/53 [==============================] - 2631s 50s/step - loss: 1.2443 - accuracy: 0.5374 - f1_score: 0.4253 - cohen_kappa: 0.2761 - MatthewsCorrelationCoefficient: 0.2777 - auc: 0.7092 - val_loss: 1.0370 - val_accuracy: 0.4656 - val_f1_score: 0.2671 - val_cohen_kappa: 0.2355 - val_MatthewsCorrelationCoefficient: 0.2766 - val_auc: 0.6652
Epoch 3/25
53/53 [==============================] - 2644s 50s/step - loss: 0.9929 - accuracy: 0.5585 - f1_score: 0.4014 - cohen_kappa: 0.3150 - MatthewsCorrelationCoefficient: 0.3163 - auc: 0.7358 - val_loss: 0.9124 - val_accuracy: 0.5725 - val_f1_score:

 50%|█████     | 1/2 [14:23:00<14:23:00, 51780.20s/it]

2022-04-07_vgg5_block_128_multiclass-b20mg-inc2
Epoch 1/25
53/53 [==============================] - 66s 1s/step - loss: 1.0749 - accuracy: 0.4395 - f1_score: 0.2631 - auc: 0.6239 - val_loss: 0.9712 - val_accuracy: 0.5267 - val_f1_score: 0.3077 - val_auc: 0.7128
Epoch 2/25
53/53 [==============================] - 63s 1s/step - loss: 0.9007 - accuracy: 0.5816 - f1_score: 0.4231 - auc: 0.7648 - val_loss: 0.8166 - val_accuracy: 0.6336 - val_f1_score: 0.4972 - val_auc: 0.8234
Epoch 3/25
53/53 [==============================] - 63s 1s/step - loss: 0.8368 - accuracy: 0.6142 - f1_score: 0.4865 - auc: 0.8005 - val_loss: 0.8570 - val_accuracy: 0.5725 - val_f1_score: 0.3988 - val_auc: 0.7794
Epoch 4/25
53/53 [==============================] - 64s 1s/step - loss: 0.7513 - accuracy: 0.6660 - f1_score: 0.5655 - auc: 0.8413 - val_loss: 0.7403 - val_accuracy: 0.6947 - val_f1_score: 0.4516 - val_auc: 0.8761
Epoch 5/25
53/53 [==============================] - 63s 1s/step - loss: 0.7297 - accuracy: 0.671

100%|██████████| 2/2 [14:49:25<00:00, 26682.68s/it]   
